<a href="https://colab.research.google.com/github/gocgodman/M2M/blob/main/M2M(mp3%20to%20midi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 셀 1: 필요한 패키지 설치 (한 번만 실행)
!pip install piano_transcription_inference
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # GPU 런타임일 경우

!pip install -q yt-dlp gdown gradio pretty_midi librosa numpy soundfile pyfluidsynth
!apt-get update -qq
!apt-get install -y -qq ffmpeg fluidsynth p7zip-full || true

In [ ]:

# Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import os, glob, shutil, zipfile, tarfile, subprocess
import gdown

# 기본 경로 설정
WORK_ROOT = "/content/ytd_pipeline_work"
os.makedirs(WORK_ROOT, exist_ok=True)

DRIVE_SF2_DIR = "/content/drive/MyDrive/sf2_library"       # 내 드라이브 sf2 폴더
DRIVE_RESULTS_DIR = "/content/drive/MyDrive/ytd_pipeline_results"
os.makedirs(DRIVE_SF2_DIR, exist_ok=True)
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)

print("WORK_ROOT:", WORK_ROOT)
print("DRIVE_SF2_DIR:", DRIVE_SF2_DIR)
print("DRIVE_RESULTS_DIR:", DRIVE_RESULTS_DIR)

# 공유 폴더 ID (필요할 때만 사용)
SHARED_FOLDER_ID = "1JkTMvPwM_XURqG2114n4Qj0rR83WEucL"
OUT_DIR = "/content/sf2_from_shared"
os.makedirs(OUT_DIR, exist_ok=True)

# 공유 폴더에서 다운로드 시도
if SHARED_FOLDER_ID:
    try:
        gdown.download_folder(id=SHARED_FOLDER_ID, output=OUT_DIR, quiet=False, use_cookies=False)
    except Exception as e:
        print("공유 폴더 다운로드 실패:", e)

# 압축 해제 함수
def try_extract_archive(path, dest):
    path_lower = path.lower()
    os.makedirs(dest, exist_ok=True)
    try:
        if path_lower.endswith(".zip"):
            with zipfile.ZipFile(path, 'r') as zf:
                zf.extractall(dest); return True
        if path_lower.endswith((".tar.gz", ".tgz", ".tar")):
            with tarfile.open(path, 'r:*') as tf:
                tf.extractall(dest); return True
        if path_lower.endswith(".7z"):
            cmd = ['7z', 'x', '-y', '-o' + dest, path]
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE); return True
    except Exception as e:
        print("압축 해제 실패:", e)
    return False

# 내 드라이브 + 공유 폴더 모두 탐색 → 압축파일 해제
for root, dirs, files in os.walk(DRIVE_SF2_DIR):
    for fn in files:
        if fn.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar", ".7z")):
            extract_dest = os.path.join(root, fn + "_extracted")
            try_extract_archive(os.path.join(root, fn), extract_dest)

for root, dirs, files in os.walk(OUT_DIR):
    for fn in files:
        if fn.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar", ".7z")):
            extract_dest = os.path.join(root, fn + "_extracted")
            try_extract_archive(os.path.join(root, fn), extract_dest)

# sf2 파일 수집
local_sf2_files = glob.glob(os.path.join(DRIVE_SF2_DIR, "**/*.sf2"), recursive=True)
local_names = {os.path.basename(f) for f in local_sf2_files}

shared_sf2_files = glob.glob(os.path.join(OUT_DIR, "**/*.sf2"), recursive=True)

print("내 드라이브 sf2 수:", len(local_sf2_files))
print("공유 드라이브 sf2 수:", len(shared_sf2_files))

# 공유 드라이브에서 내 드라이브에 없는 파일만 복사
copied = []
for p in shared_sf2_files:
    fname = os.path.basename(p)
    if fname in local_names:
        print("이미 존재 → 건너뜀:", fname)
        continue
    dest = os.path.join(DRIVE_RESULTS_DIR, fname)
    try:
        shutil.copy(p, dest)
        copied.append(dest)
    except Exception as e:
        print("복사 실패:", p, e)

print("새로 복사된 sf2 수:", len(copied))
for f in copied:
    print(" -", f)

In [ ]:

# 통합 셀: ByteDance 전사 + 경량 페달 검출 통합 버전
# 실행 환경: Colab (GPU 권장). 사전 설치: piano_transcription_inference, torch, librosa, pretty_midi 등.

import os, sys, time, uuid, json, glob, shutil, zipfile, subprocess, traceback
from pathlib import Path

import numpy as np
import librosa
import pretty_midi
from scipy.ndimage import uniform_filter1d, binary_closing

# Torch는 경량 페달 모델(선택적)에서 사용
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------------- 결과/작업 폴더 ----------------
DRIVE_RESULTS_DIR = "/content/drive/MyDrive/ytd_pipeline_results"
WORK_ROOT = "/content/ytd_pipeline_work"
STATE_FILE = os.path.join(DRIVE_RESULTS_DIR, "pipeline_state.json")
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)
os.makedirs(WORK_ROOT, exist_ok=True)

# ---------------- 유틸리티 ----------------
def run_cmd(cmd, check=False):
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if check and proc.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\nSTDOUT:{proc.stdout}\nSTDERR:{proc.stderr}")
    return proc.returncode, proc.stdout, proc.stderr

def remove_extension(filepath):
    return ".".join(os.path.basename(filepath).split('.')[:-1])

# ---------------- ByteDance 전사 (piano_transcription_inference) ----------------
# 설치: !pip install piano_transcription_inference
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio
_transcriptor = PianoTranscription(device='cuda' if torch.cuda.is_available() else 'cpu')

def transcribe_file(input_path, outfolder='.'):
    """
    piano_transcription_inference 모델로 오디오 -> MIDI 변환.
    outfolder에 mid 파일 생성 후 경로 반환.
    """
    base = remove_extension(input_path)
    out_mid = os.path.join(outfolder, base + ".mid")
    audio, _ = load_audio(input_path, sr=sample_rate)
    _transcriptor.transcribe(audio, out_mid)
    return out_mid

# ---------------- 기존 yt-dlp 헬퍼 ----------------
def expand_playlist_to_video_urls(playlist_url):
    cmd = ["yt-dlp", "--flat-playlist", "-J", playlist_url]
    code, out, err = run_cmd(cmd)
    if code != 0:
        raise RuntimeError(f"yt-dlp playlist expand failed: {err}")
    j = json.loads(out)
    entries = j.get("entries", [])
    urls = []
    for e in entries:
        vid = e.get("id")
        if vid:
            urls.append(f"https://www.youtube.com/watch?v={vid}")
    return urls

def download_youtube_audio_single(url, outdir, fmt="mp3", audio_bitrate="192K"):
    os.makedirs(outdir, exist_ok=True)
    out_template = os.path.join(outdir, "%(playlist_index)s-%(title)s.%(ext)s")
    cmd = ["yt-dlp", "-x", "--audio-format", fmt, "--audio-quality", audio_bitrate, "-o", out_template, url]
    code, out, err = run_cmd(cmd)
    files = sorted(glob.glob(os.path.join(outdir, f"*.{fmt}")))
    if not files:
        raise RuntimeError(f"No audio file produced by yt-dlp for {url}. yt-dlp stderr: {err}")
    return files[-1]

# ---------------- 기존 페달 검출(규칙 기반) ----------------
def compute_spectral_flux(y, sr, hop_length, n_fft=2048):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_norm = S / (np.sum(S, axis=0, keepdims=True) + 1e-8)
    flux = np.sqrt(np.sum(np.diff(S_norm, axis=1).clip(min=0)**2, axis=0))
    return np.concatenate(([0.0], flux))

def detect_pedal_rule(audio_path,
                      sr=22050,
                      hop_length=512,
                      low_freq_cut=500,
                      energy_smooth=0.5,
                      on_z=1.0,
                      off_z=0.7,
                      min_event_len=0.06,
                      merge_gap=0.06,
                      closing_size=3):
    """
    규칙 기반 페달 검출: 빠르고 가벼움. (기본 동작)
    """
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    frame_energy = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop_length)[0]
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)
    low_idx = np.where(freqs <= low_freq_cut)[0]
    low_energy = S[low_idx, :].sum(axis=0) if len(low_idx) > 0 else np.zeros_like(frame_energy)
    e = frame_energy / (frame_energy.max() + 1e-8)
    le = low_energy / (low_energy.max() + 1e-8) if low_energy.max() > 0 else low_energy
    combined = 0.6 * le + 0.4 * e
    window = int(max(1, energy_smooth * (sr / hop_length)))
    combined_smooth = uniform_filter1d(combined, size=window)
    mu = combined_smooth.mean(); sigma = combined_smooth.std() + 1e-8
    on_thr = mu + on_z * sigma; off_thr = mu + off_z * sigma
    mask = np.zeros_like(combined_smooth, dtype=bool)
    state = False
    for i, v in enumerate(combined_smooth):
        if not state and v >= on_thr:
            state = True; mask[i] = True
        elif state:
            mask[i] = True
            if v < off_thr:
                state = False
    if closing_size > 1:
        mask = binary_closing(mask, structure=np.ones(closing_size))
    times = librosa.frames_to_time(np.arange(len(mask)), sr=sr, hop_length=hop_length)
    events = []; prev=False; start=None
    for t,m in zip(times, mask):
        if m and not prev: start=t
        if (not m) and prev and start is not None:
            events.append((start,t)); start=None
        prev=m
    if prev and start is not None: events.append((start,times[-1]))
    filtered=[]
    for s,e in events:
        if (e-s) >= min_event_len:
            if filtered and s - filtered[-1][1] <= merge_gap:
                filtered[-1] = (filtered[-1][0], e)
            else:
                filtered.append((s,e))
    return filtered

# ---------------- 경량 PedalCNN (선택적 보조 모델) ----------------
class PedalCNN(nn.Module):
    def __init__(self, in_ch=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_ch, 16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.mean(dim=2)
        x = self.fc(x)
        return torch.sigmoid(x).squeeze(-1)

def extract_frame_features(y, sr=22050, hop_length=512, n_mels=40):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=hop_length, n_mels=n_mels)
    logS = librosa.power_to_db(S, ref=np.max)
    return logS  # (n_mels, T)

def predict_pedal_with_model(audio_path, model, device='cpu', sr=22050, hop_length=512, threshold=0.5):
    """
    간단 보조 추론: 모델이 전체 파일에 대해 페달 존재를 판단하면 규칙 기반으로 구간 추출.
    (모델을 프레임 단위로 출력하도록 바꾸면 더 정교하게 사용 가능)
    """
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    feats = extract_frame_features(y, sr=sr, hop_length=hop_length)  # (n_mels, T)
    X = torch.tensor(feats[np.newaxis, :, :], dtype=torch.float32)  # (1, n_mels, T)
    X = X.mean(dim=1, keepdim=True)  # (1,1,T)
    model.to(device); model.eval()
    with torch.no_grad():
        prob = float(model(X.to(device)).cpu().numpy())
    if prob >= threshold:
        events = detect_pedal_rule(audio_path, sr=sr, hop_length=hop_length)
    else:
        events = []
    return events

def detect_pedal_lightweight(audio_path,
                             use_model=False,
                             model_path=None,
                             device='cpu',
                             sr=22050,
                             hop_length=512,
                             rule_params=None,
                             model_threshold=0.5):
    """
    통합 페달 검출기: 규칙 기반 우선, 필요시 경량 모델로 보강.
    반환: [(start,end), ...]
    """
    if rule_params is None:
        rule_params = {}
    rule_events = detect_pedal_rule(audio_path, sr=sr, hop_length=hop_length, **rule_params)
    if not use_model or model_path is None:
        return rule_events
    try:
        model = PedalCNN(in_ch=1)
        ckpt = torch.load(model_path, map_location=device)
        if isinstance(ckpt, dict) and 'state_dict' in ckpt:
            model.load_state_dict(ckpt['state_dict'])
        else:
            model.load_state_dict(ckpt)
        model_events = predict_pedal_with_model(audio_path, model, device=device, hop_length=hop_length, threshold=model_threshold)
        combined = rule_events.copy()
        for me in model_events:
            if not any((abs(me[0]-re[0])<0.05 and abs(me[1]-re[1])<0.05) for re in combined):
                combined.append(me)
        combined = sorted(combined, key=lambda x: x[0])
        merged=[]
        for s,e in combined:
            if not merged:
                merged.append([s,e])
            else:
                if s <= merged[-1][1] + 0.05:
                    merged[-1][1] = max(merged[-1][1], e)
                else:
                    merged.append([s,e])
        return [(s,e) for s,e in merged]
    except Exception:
        return rule_events

# ---------------- MIDI에 페달 CC 삽입 ----------------
def insert_pedal_cc_into_midi(midi_in_path, midi_out_path, pedal_events, piano_program=0):
    pm = pretty_midi.PrettyMIDI(midi_in_path)
    for inst in pm.instruments:
        inst.program = piano_program
    target_inst = pm.instruments[0] if pm.instruments else pretty_midi.Instrument(program=piano_program)
    if not pm.instruments:
        pm.instruments.append(target_inst)
    for (s,e) in pedal_events:
        on_time = max(0.0, s - 0.02); off_time = e + 0.02
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=127, time=on_time))
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=0, time=off_time))
    for inst in pm.instruments:
        inst.control_changes.sort(key=lambda cc: cc.time)
    pm.write(midi_out_path)

# ---------------- fluidsynth 렌더링 ----------------
def render_midi_to_wav(midi_path, wav_out_path, sf2_path, sample_rate=44100):
    if sf2_path is None or not os.path.exists(sf2_path):
        return None
    try:
        cmd = ['fluidsynth', '-ni', sf2_path, midi_path, '-F', wav_out_path, '-r', str(sample_rate)]
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return wav_out_path
    except Exception:
        return None

# ---------------- 체크포인트 ----------------
def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"processed": [], "failed": [], "items": {}, "last_update": None}

def save_state(state):
    state["last_update"] = time.time()
    with open(STATE_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=2)

# ---------------- 항목 처리 (통합) ----------------
def process_item_from_url_or_path(item, sf2_path=None, tmp_dir=None, pedal_model_path=None, use_pedal_model=False):
    """
    item: YouTube URL 또는 로컬 오디오 파일 경로
    pedal_model_path: (선택) 경량 페달 모델 체크포인트 경로
    use_pedal_model: 모델 보조 사용 여부
    """
    if tmp_dir is None:
        tmp_dir = os.path.join(WORK_ROOT, "tmp_" + uuid.uuid4().hex)
    os.makedirs(tmp_dir, exist_ok=True)
    try:
        # 1) 오디오 확보
        if isinstance(item, str) and item.startswith("http"):
            mp3 = download_youtube_audio_single(item, tmp_dir, fmt="mp3")
        else:
            mp3 = item

        # 2) 전사 (ByteDance)
        mid = transcribe_file(mp3, outfolder=tmp_dir)

        # 3) 페달 검출 (규칙 기반 또는 모델 보조)
        pedals = detect_pedal_lightweight(mp3, use_model=use_pedal_model, model_path=pedal_model_path, device='cuda' if torch.cuda.is_available() else 'cpu')

        # 4) MIDI에 CC 삽입
        mid_pedal = os.path.join(tmp_dir, os.path.splitext(os.path.basename(mid))[0] + "_pedal.mid")
        insert_pedal_cc_into_midi(mid, mid_pedal, pedals, piano_program=0)

        # 5) Drive에 저장
        saved_mp3 = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(mp3)); shutil.copy(mp3, saved_mp3)
        saved_midi = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(mid_pedal)); shutil.copy(mid_pedal, saved_midi)

        # 6) 렌더링 (선택)
        saved_wav = None
        if sf2_path:
            wav_out = os.path.join(tmp_dir, os.path.splitext(os.path.basename(mp3))[0] + "_render.wav")
            rendered = render_midi_to_wav(mid_pedal, wav_out, sf2_path)
            if rendered:
                saved_wav = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(wav_out)); shutil.copy(wav_out, saved_wav)

        return {"status":"ok", "mp3": saved_mp3, "midi": saved_midi, "wav": saved_wav, "pedals": pedals}
    except Exception as e:
        return {"status":"error", "error": str(e), "trace": traceback.format_exc()}

# ---------------- 파일 업로드 처리 ----------------
def process_files(gr_files,
                  sf2_choice,
                  uploaded_sf2,
                  use_pedal_model=False,
                  pedal_model_path=None,
                  sustain_tolerance=0.2,
                  pedal_energy_smooth=0.5,
                  pedal_low_freq_cut=500,
                  pedal_low_energy_weight=0.6,
                  pedal_flux_weight=0.4,
                  pedal_on_threshold=1.0,
                  pedal_off_threshold=0.7,
                  pedal_min_event_len=0.08,
                  pedal_merge_gap=0.08):
    """
    Gradio 파일 업로드용 래퍼. use_pedal_model, pedal_model_path 인자를 통해 모델 보조 사용 가능.
    """
    chosen_sf2 = None
    if sf2_choice and sf2_choice != "None":
        chosen_sf2 = sf2_choice
    if uploaded_sf2:
        if isinstance(uploaded_sf2, list) and len(uploaded_sf2) > 0:
            up = uploaded_sf2[0]
            chosen_sf2 = up['name'] if isinstance(up, dict) and 'name' in up else up
    if not gr_files:
        return None, None
    work_dir = "/tmp/transcribe_" + uuid.uuid4().hex
    os.makedirs(work_dir, exist_ok=True)
    local_paths = []
    for f in gr_files:
        if isinstance(f, dict) and 'name' in f:
            local_paths.append(f['name'])
        else:
            local_paths.append(f)
    midi_outs = []
    for audio_path in local_paths:
        try:
            res = process_item_from_url_or_path(audio_path, sf2_path=chosen_sf2, tmp_dir=work_dir, pedal_model_path=pedal_model_path, use_pedal_model=use_pedal_model)
            if res.get("status") != "ok":
                return f"처리 실패: {res.get('error')}", None
            midi_outs.append(res.get("midi"))
        except Exception as e:
            return f"처리 중 예외: {e}", None
    if len(midi_outs) == 1:
        final_midi = midi_outs[0]
    else:
        zip_path = os.path.join(work_dir, "results_with_pedal.zip")
        with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
            for p in midi_outs:
                zf.write(p, arcname=os.path.basename(p))
        final_midi = zip_path
    wav_candidates = sorted(glob.glob(os.path.join(DRIVE_RESULTS_DIR, "*_render.wav")))
    wav_path = wav_candidates[-1] if wav_candidates else None
    return final_midi, wav_path

# ---------------- 재생목록 제너레이터 ----------------
def playlist_pipeline_generator(playlist_text, sf2_choice_path, use_pedal_model=False, pedal_model_path=None):
    missing = []
    for name in ("expand_playlist_to_video_urls", "process_item_from_url_or_path", "load_state", "save_state"):
        if name not in globals():
            missing.append(name)
    if missing:
        yield f"필수 함수가 정의되어 있지 않습니다: {', '.join(missing)}", None
        return
    lines = [ln.strip() for ln in str(playlist_text).splitlines() if ln.strip()]
    items = []
    for ln in lines:
        if ln.startswith("http") and ("playlist" in ln or "list=" in ln):
            yield f"재생목록 확장 중: {ln}", None
            try:
                urls = expand_playlist_to_video_urls(ln)
                if not urls:
                    yield f"재생목록에서 URL을 찾지 못했습니다: {ln}", None
                else:
                    items.extend(urls)
                    yield f"재생목록에서 {len(urls)}개 항목 발견", None
            except Exception as e:
                yield f"재생목록 확장 실패: {e}", None
        else:
            items.append(ln)
    total = len(items)
    if total == 0:
        yield "처리할 항목이 없습니다.", None
        return
    try:
        state = load_state()
    except Exception as e:
        yield f"상태 파일 로드 실패: {e}", None
        return
    yield f"총 항목: {total}. 이미 처리된 항목: {len(state.get('processed', []))}. 처리 시작합니다...", None
    for idx, item in enumerate(items, start=1):
        if item in state.get("processed", []):
            yield f"[{idx}/{total}] 이미 처리됨: {item}", None
            continue
        yield f"[{idx}/{total}] 다운로드/전사 시작: {item}", None
        try:
            res = process_item_from_url_or_path(item, sf2_path=sf2_choice_path, pedal_model_path=pedal_model_path, use_pedal_model=use_pedal_model)
        except Exception as e:
            res = {"status": "error", "error": str(e)}
        if res.get("status") == "ok":
            state.setdefault("processed", []).append(item)
            state.setdefault("items", {})[item] = {
                "mp3": res.get("mp3"),
                "midi": res.get("midi"),
                "wav": res.get("wav"),
                "time": time.time()
            }
            try:
                save_state(state)
            except Exception as e:
                yield f"[{idx}/{total}] 완료했으나 상태 저장 실패: {e}", res
                continue
            yield f"[{idx}/{total}] 완료: {item} → MIDI: {os.path.basename(res.get('midi') or '')} WAV: {os.path.basename(res.get('wav') or '') or '없음'}", res
        else:
            state.setdefault("failed", []).append({"item": item, "error": res.get("error")})
            try:
                save_state(state)
            except Exception as e:
                yield f"[{idx}/{total}] 실패: {item} 에러 저장 실패: {e}", None
                continue
            yield f"[{idx}/{total}] 실패: {item} 에러: {res.get('error')}", None
    yield "모든 항목 처리 완료(또는 건너뛰기 완료). Drive의 pipeline_state.json을 확인하세요.", None

print("통합 완료: ByteDance 전사 + 경량 페달 검출(규칙 기반 + 선택적 모델 보조) 파이프라인이 준비되었습니다.")

In [ ]:

# 완전한 통합 UI 셀 — 업로드 탭 + 재생목록 탭, SF2 선택 포함 (단일 셀로 실행)
# 이 셀은 이전에 정의된 함수들(process_files, playlist_pipeline_generator 등)을 사용합니다.
# 만약 해당 함수들이 아직 정의되지 않았다면 먼저 관련 셀들을 실행하세요.

import gradio as gr
import os, traceback

# sf2_files가 이미 존재하면 사용, 아니면 빈 리스트
try:
    sf2_choices = ["None"] + sf2_files
except Exception:
    sf2_choices = ["None"]

def get_sf2_choice_value(choice, uploaded_sf2):
    """
    UI에서 선택된 SF2 경로를 결정:
    - uploaded_sf2가 있으면 업로드된 파일(첫번째)을 우선 사용
    - 아니면 드롭다운 선택(choice)를 사용
    """
    chosen = None
    try:
        if uploaded_sf2 and isinstance(uploaded_sf2, list) and len(uploaded_sf2) > 0:
            up = uploaded_sf2[0]
            if isinstance(up, dict) and 'name' in up:
                chosen = up['name']
            else:
                chosen = up
    except Exception:
        chosen = None
    if not chosen and choice and choice != "None":
        chosen = choice
    return chosen

# 제너레이터 래퍼: 업로드 우선 SF2 결정 후 내부 제너레이터에 위임
def playlist_wrapper_with_upload(playlist_text, sf2_choice_val, uploaded_sf2_val, sf2_direct_input):
    try:
        chosen = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val)
        if sf2_direct_input:
            chosen = sf2_direct_input
        # playlist_pipeline_generator는 제너레이터여야 함
        yield from playlist_pipeline_generator(playlist_text, chosen)
    except Exception as e:
        # 에러 발생 시 사용자에게 즉시 스트리밍 형태로 알림
        yield f"재생목록 처리 중 예외 발생: {e}", None

# 업로드 처리 콜백(동기 함수)
def on_run_upload(gr_files, sf2_choice_val, uploaded_sf2_val,
                  sustain_tolerance, pedal_energy_smooth, pedal_low_freq_cut,
                  pedal_low_energy_weight, pedal_flux_weight,
                  pedal_on_threshold, pedal_off_threshold,
                  pedal_min_event_len, pedal_merge_gap):
    try:
        chosen_sf2 = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val)
        status = f"업로드 처리 시작. SF2: {os.path.basename(chosen_sf2) if chosen_sf2 else '없음'}"
        final_midi, wav_path = process_files(
            gr_files,
            sf2_choice=sf2_choice_val,
            uploaded_sf2=uploaded_sf2_val,
            sustain_tolerance=sustain_tolerance,
            pedal_energy_smooth=pedal_energy_smooth,
            pedal_low_freq_cut=pedal_low_freq_cut,
            pedal_low_energy_weight=pedal_low_energy_weight,
            pedal_flux_weight=pedal_flux_weight,
            pedal_on_threshold=pedal_on_threshold,
            pedal_off_threshold=pedal_off_threshold,
            pedal_min_event_len=pedal_min_event_len,
            pedal_merge_gap=pedal_merge_gap
        )
        return status, final_midi, wav_path
    except Exception as e:
        tb = traceback.format_exc()
        return f"업로드 처리 중 오류: {e}\n{tb}", None, None

# Blocks UI 생성 및 실행
with gr.Blocks() as combined_demo:
    gr.Markdown("## 통합: 파일 업로드 + 재생목록 배치 처리 (공통 SF2 선택)")
    with gr.Row():
        with gr.Column(scale=1):
            sf2_dropdown = gr.Dropdown(choices=sf2_choices, value=sf2_choices[0], label="Drive에서 선택할 SF2")
            sf2_upload = gr.Files(file_types=[".sf2"], label="또는 SF2 업로드 (선택)")
            gr.Markdown("**설명:** 업로드된 SF2가 있으면 업로드된 파일을 우선 사용합니다. 드롭다운에서 Drive에 복사된 SF2를 선택할 수도 있습니다.")
        with gr.Column(scale=2):
            status_box = gr.Textbox(label="전역 상태", interactive=False)
            status_box.value = f"결과 폴더: {DRIVE_RESULTS_DIR if 'DRIVE_RESULTS_DIR' in globals() else '설정 필요'}"

    with gr.Tabs():
        with gr.TabItem("파일 업로드 처리"):
            upload_files = gr.Files(file_types=[".wav", ".mp3"], label="오디오 파일 업로드")
            sustain = gr.Slider(0.05, 0.6, value=0.2, step=0.01, label="Sustain tolerance (note merge, sec)")
            pedal_smooth = gr.Slider(0.1, 1.0, value=0.5, step=0.05, label="Pedal energy smoothing (sec)")
            pedal_lowcut = gr.Slider(200, 2000, value=500, step=50, label="Pedal low-frequency cutoff (Hz)")
            pedal_low_w = gr.Slider(0.0, 1.0, value=0.6, step=0.05, label="Low-energy weight (pedal)")
            pedal_flux_w = gr.Slider(0.0, 1.0, value=0.4, step=0.05, label="Flux weight (pedal)")
            pedal_on = gr.Slider(0.2, 2.0, value=1.0, step=0.1, label="Pedal ON threshold (sigma multiplier)")
            pedal_off = gr.Slider(0.0, 1.5, value=0.7, step=0.05, label="Pedal OFF threshold (sigma multiplier)")
            pedal_min = gr.Slider(0.02, 0.5, value=0.08, step=0.01, label="Min pedal event length (sec)")
            pedal_merge = gr.Slider(0.01, 0.3, value=0.08, step=0.01, label="Merge gap for pedal events (sec)")
            run_upload = gr.Button("업로드 파일 처리 시작")
            upload_result_file = gr.File(label="Download MIDI or ZIP")
            upload_preview = gr.Audio(label="Preview (WAV) - rendered with chosen SF2", type="filepath")

            run_upload.click(
                fn=on_run_upload,
                inputs=[upload_files, sf2_dropdown, sf2_upload,
                        sustain, pedal_smooth, pedal_lowcut, pedal_low_w, pedal_flux_w,
                        pedal_on, pedal_off, pedal_min, pedal_merge],
                outputs=[status_box, upload_result_file, upload_preview]
            )

        with gr.TabItem("재생목록 배치 처리"):
            playlist_input = gr.Textbox(lines=6, label="재생목록 URL 또는 영상 URL들 (줄바꿈으로 여러개 입력)")
            playlist_sf2_input = gr.Textbox(label="(선택) SF2 경로를 직접 입력하거나 위 드롭다운/업로드 사용")
            run_playlist_btn = gr.Button("재생목록 처리 시작")
            playlist_log = gr.Textbox(label="진행 로그")
            playlist_last = gr.JSON(label="마지막 항목 결과")

            # 제너레이터 래퍼를 직접 바인딩 (Gradio는 제너레이터 함수를 스트리밍으로 처리)
            run_playlist_btn.click(
                fn=playlist_wrapper_with_upload,
                inputs=[playlist_input, sf2_dropdown, sf2_upload, playlist_sf2_input],
                outputs=[playlist_log, playlist_last]
            )

    gr.Markdown("**사용법:** 상단에서 SF2를 드롭다운으로 선택하거나 SF2 파일을 업로드하세요. 업로드 탭은 로컬 파일을 처리하고, 재생목록 탭은 유튜브 재생목록/URL을 배치 처리합니다. 처리 결과는 Drive의 ytd_pipeline_results에 저장됩니다.")

# 실행 (Colab에서는 share=True, debug=False 권장)
combined_demo.launch(share=True, debug=False)